## INFO 2950 Group Project Appendix

Nichole Zheng (nz268) & Hedy Song (ys565)

### Chosen Dataset:
The dataset is a collection of daily records of the top trending videos on YouTube across multiple countries. It contains metadata such as video titles, channel names, publish times, tags, views, likes, dislikes, comment counts, and descriptions. Each region’s data is stored in a separate CSV file, and accompanying JSON files provide mappings of category IDs to video categories. 

It can be found at: https://www.kaggle.com/datasets/datasnaek/youtube-new/data

In terms of availability, the dataset is freely available and downloadable from Kaggle at the above link. It is provided in CSV format, organized by country. It was originally collected using the YouTube Data API (documentation: YouTube Data API v3). 

Below is the column names and their corresponding description **before cleaning**

- **video_id：** Unique identifier for each YouTube video.
- **trending_date:** Date when the video appeared on the trending list (YYYY-MM-DD format).
- **title:** Title of the YouTube video.
- **channel_title:** Name of the channel that uploaded the video.
- **category_id:** Numeric ID representing the video’s category (e.g., entertainment, music, news).
- **publish_time:** Exact timestamp when the video was originally published on YouTube (UTC).
- **tags:** List of keywords or tags associated with the video.
- **views:** Total number of views the video had when it was trending.
- **likes:** Number of likes the video received.
- **dislikes:** Number of dislikes the video received.
- **comment_count:** Number of comments posted on the video.
- **thumbnail_link:** URL of the video’s thumbnail image.
- **comments_disabled:** Boolean flag indicating whether comments were disabled (True/False).
- **ratings_disabled:** Boolean flag showing whether viewers were prevented from rating (liking/disliking) the video.
- **video_error_or_removed:** Boolean flag showing whether the video was later removed or produced an error.
- **description:** Text description provided by the uploader. May contain keywords, links, or summaries of the video’s content.


In [1]:
import pandas as pd
from datetime import datetime
import json
import matplotlib.pyplot as plt

## A. Data Cleaning for the United Kingdom

In [2]:
# Load the dataset
df = pd.read_csv("GBvideos.csv", encoding='utf-8')
print(df.head())

      video_id trending_date  \
0  Jw1Y-zhQURU      17.14.11   
1  3s1rvMFUweQ      17.14.11   
2  n1WpP7iowLc      17.14.11   
3  PUTEiSjKwJU      17.14.11   
4  rHwDegptbI4      17.14.11   

                                               title  \
0      John Lewis Christmas Ad 2017 - #MozTheMonster   
1          Taylor Swift: …Ready for It? (Live) - SNL   
2         Eminem - Walk On Water (Audio) ft. Beyoncé   
3  Goals from Salford City vs Class of 92 and Fri...   
4  Dashcam captures truck's near miss with child ...   

                channel_title  category_id              publish_time  \
0                  John Lewis           26  2017-11-10T07:38:29.000Z   
1         Saturday Night Live           24  2017-11-12T06:24:44.000Z   
2                  EminemVEVO           10  2017-11-10T17:00:03.000Z   
3  Salford City Football Club           17  2017-11-13T02:30:38.000Z   
4            Cute Girl Videos           25  2017-11-13T01:45:13.000Z   

                                     

#### 1. Handle Duplicates, Missing Values, and Invalid Entries

Result:

- The dataset contains no duplicates.
- The description column has 612 missing values.
- Any negative values in views, likes, dislikes, or comment_count (if present) were removed.

In [3]:
# Check for duplicates
df.duplicated().sum()
# check for missing values
df.isna().sum()

video_id                    0
trending_date               0
title                       0
channel_title               0
category_id                 0
publish_time                0
tags                        0
views                       0
likes                       0
dislikes                    0
comment_count               0
thumbnail_link              0
comments_disabled           0
ratings_disabled            0
video_error_or_removed      0
description               612
dtype: int64

In [4]:
# Remove invalid values
df = df[(df['views'] >= 0) & (df['likes'] >= 0) & (df['dislikes'] >= 0) & (df['comment_count'] >= 0)]

#### 2. Drop Unnecessary Columns

This dataset contains no duplicates, though there are a few null values in the description column. We will be **dropping the description column** because:
1. Many **unique strings** of dataset making it hard to compare
2. Only a **few null values** therefore can't do binary variable to compare if having versus not having a description is a factor that decides whether a video goes viral 

The **thumbnail_link column** is also dropped, as extracting and analyzing cover images with an LLM would be too resource-intensive. Instead, we will focus our research on other variables, such as title and tags.

In [5]:
df = df.drop(['thumbnail_link','description'], axis=1)

#### 3. Date Conversion

Converted trending_date and publish_time to **standard datetime formats** and **removed the timezone information** to ensure consistency across all records.

All datetime are now in the format: YYYY-MM-DD (ex: 2017-11-14)

In [6]:
df['trending_date'] = pd.to_datetime(df['trending_date'], format='%y.%d.%m', errors='coerce')
df['publish_time'] = pd.to_datetime(df['publish_time'], errors='coerce')

# Ensure both datetime columns are timezone-naive (remove timezone info)
df['publish_time'] = pd.to_datetime(df['publish_time'], utc=True).dt.tz_localize(None)
df['trending_date'] = pd.to_datetime(df['trending_date'], format='%y.%d.%m', errors='coerce')

#### 4. Map the corresponding category ID to category name

Initially, GB_videos.csv contained all video information, while a separate JSON file mapped category IDs (int64) to their corresponding names. To efficiently analyze videos by category, we replaced each category ID with its category name.

In [7]:
# Load JSON file
with open("GB_category_id.json", "r") as f:
    data = json.load(f)

# Extract mapping: category_id → category_name
category_mapping = {int(item['id']): item['snippet']['title'] for item in data['items']}

# Map category names
df['category_name'] = df['category_id'].map(category_mapping)

#### 5. Clean tags and count number of tags per video

To better understand video metadata and facilitate analysis, we cleaned the tags column by **replacing placeholder values** like [none] and **filling missing entries**. We then created a new column, **Num_tags**, to count the number of tags associated with each video. This step is necessary because the **number of tags** can be an important feature for understanding video categorization, popularity, and searchability.

In [8]:
# Count number of tags per video directly
df['Num_tags'] = (
    df['tags']
    .replace('[none]', '')            # replace '[none]' with empty string
    .fillna('')                       # handle missing values
    .apply(lambda x: 0 if x == '' else len(x.split('|')))  # count tags
)

# Quick check
df[['tags', 'Num_tags']].head()

,tags,Num_tags
0,"christmas|""john lewis christmas""|""john lewis""|...",9
1,"SNL|""Saturday Night Live""|""SNL Season 43""|""Epi...",36
2,"Eminem|""Walk""|""On""|""Water""|""Aftermath/Shady/In...",6
3,"Salford City FC|""Salford City""|""Salford""|""Clas...",9
4,[none],0


#### 6. Drop comments_disabled, ratings_disabled and video_error_or_removed columns (qianmian)

This checks whether it’s useful (e.g., if almost all are False, maybe drop it).

Since less than 2% of the data for these boolean columns are False, we decided to not use these columns in our anaylsis and drop them.

In [9]:
# Frequency table for comments_disabled, ratings_disabled and video_error_or_removed columns

freq_booleans = pd.DataFrame({
    'comments_disabled': df['comments_disabled'].value_counts(),
    'ratings_disabled': df['ratings_disabled'].value_counts(),
    'video_error_or_removed': df['video_error_or_removed'].value_counts()
}).fillna(0).astype(int)

print("Combined frequency table for columns:")
print(freq_booleans)

freq_percent = df[['comments_disabled', 'ratings_disabled', 'video_error_or_removed']] \
    .apply(lambda col: col.value_counts(normalize=True) * 100) \
    .round(2)

print("Percentage of True/False for each column:")
print(freq_percent)

Combined frequency table for columns:
       comments_disabled  ratings_disabled  video_error_or_removed
False              38233             38644                   38847
True                 683               272                      69
Percentage of True/False for each column:
       comments_disabled  ratings_disabled  video_error_or_removed
False              98.24              99.3                   99.82
True                1.76               0.7                    0.18


In [10]:
# Columns are dropped
df = df.drop(['comments_disabled', 'ratings_disabled', 'video_error_or_removed'], axis=1)

In [11]:
# Prints updated dataframe
print("Updated DataFrame (after dropping boolean columns):")
df.info()
print("\nPreview of updated DataFrame:")
print(df.head())

Updated DataFrame (after dropping boolean columns):
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38916 entries, 0 to 38915
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   video_id       38916 non-null  object        
 1   trending_date  38916 non-null  datetime64[ns]
 2   title          38916 non-null  object        
 3   channel_title  38916 non-null  object        
 4   category_id    38916 non-null  int64         
 5   publish_time   38916 non-null  datetime64[ns]
 6   tags           38916 non-null  object        
 7   views          38916 non-null  int64         
 8   likes          38916 non-null  int64         
 9   dislikes       38916 non-null  int64         
 10  comment_count  38916 non-null  int64         
 11  category_name  38826 non-null  object        
 12  Num_tags       38916 non-null  int64         
dtypes: datetime64[ns](2), int64(6), object(5)
memory usage: 3.9+ MB

Prev

In [12]:
# Makes new cleaned CSV file
df.to_csv("cleaned_UKvideos_all.csv", index=False)

#### NOTE

Called df.info() to check that all columns' dtypes match what they are supposed to be, and printed the dataset for checking.

---

## B. Data Cleaning for the United States

In [13]:
# Load the dataset
df_US = pd.read_csv("USvideos.csv", encoding='utf-8')
print(df_US.head())

      video_id trending_date  \
0  2kyS6SvSYSE      17.14.11   
1  1ZAPwfrtAFY      17.14.11   
2  5qpjK5DgCt4      17.14.11   
3  puqaWrEC7tY      17.14.11   
4  d380meD0W0M      17.14.11   

                                               title          channel_title  \
0                 WE WANT TO TALK ABOUT OUR MARRIAGE           CaseyNeistat   
1  The Trump Presidency: Last Week Tonight with J...        LastWeekTonight   
2  Racist Superman | Rudy Mancuso, King Bach & Le...           Rudy Mancuso   
3                   Nickelback Lyrics: Real or Fake?  Good Mythical Morning   
4                           I Dare You: GOING BALD!?               nigahiga   

   category_id              publish_time  \
0           22  2017-11-13T17:13:01.000Z   
1           24  2017-11-13T07:30:00.000Z   
2           23  2017-11-12T19:05:24.000Z   
3           24  2017-11-13T11:00:04.000Z   
4           24  2017-11-12T18:01:41.000Z   

                                                tags    views   lik

#### 1. Handle Duplicates, Missing Values, and Invalid Entries

Result:

- The dataset contains no duplicates.
- The description column has 570 missing values.
- Any negative values in views, likes, dislikes, or comment_count (if present) were removed.

In [14]:
# Check for duplicates
df_US.duplicated().sum()
# check for missing values
df_US.isna().sum()

video_id                    0
trending_date               0
title                       0
channel_title               0
category_id                 0
publish_time                0
tags                        0
views                       0
likes                       0
dislikes                    0
comment_count               0
thumbnail_link              0
comments_disabled           0
ratings_disabled            0
video_error_or_removed      0
description               570
dtype: int64

In [15]:
# Remove invalid values
df_US = df_US[(df_US['views'] >= 0) & (df_US['likes'] >= 0) & (df_US['dislikes'] >= 0) & (df_US['comment_count'] >= 0)]

#### 2. Drop Unnecessary Columns

This dataset contains no duplicates, though there are a few null values in the description column. We will be **dropping the description column** because:
1. Many **unique strings** of dataset making it hard to compare
2. Only a **few null values** therefore can't do binary variable to compare if having versus not having a description is a factor that decides whether a video goes viral 

The **thumbnail_link column** is also dropped, as extracting and analyzing cover images with an LLM would be too resource-intensive. Instead, we will focus our research on other variables, such as title and tags.

In [16]:
df_US = df_US.drop(['thumbnail_link','description'], axis=1)

#### 3. Date Conversion

Converted trending_date and publish_time to **standard datetime formats** and **removed the timezone information** to ensure consistency across all records.

All datetime are now in the format: YYYY-MM-DD (ex: 2017-11-14)

In [17]:
df_US['trending_date'] = pd.to_datetime(df_US['trending_date'], format='%y.%d.%m', errors='coerce')
df_US['publish_time'] = pd.to_datetime(df_US['publish_time'], errors='coerce')

# Ensure both datetime columns are timezone-naive (remove timezone info)
df_US['publish_time'] = pd.to_datetime(df_US['publish_time'], utc=True).dt.tz_localize(None)
df_US['trending_date'] = pd.to_datetime(df_US['trending_date'], format='%y.%d.%m', errors='coerce')

#### 4. Map the corresponding category ID to category name

Initially, US_videos.csv contained all video information, while a separate JSON file mapped category IDs (int64) to their corresponding names. To efficiently analyze videos by category, we replaced each category ID with its category name.

In [18]:
# Load JSON file
with open("US_category_id.json", "r") as f:
    data = json.load(f)

# Extract mapping: category_id → category_name
category_mapping = {int(item['id']): item['snippet']['title'] for item in data['items']}

# Map category names
df_US['category_name'] = df_US['category_id'].map(category_mapping)

#### 5. Clean tags and count number of tags per video

To better understand video metadata and facilitate analysis, we cleaned the tags column by **replacing placeholder values** like [none] and **filling missing entries**. We then created a new column, **Num_tags**, to count the number of tags associated with each video. This step is necessary because the **number of tags** can be an important feature for understanding video categorization, popularity, and searchability.

In [19]:
# Count number of tags per video directly
df_US['Num_tags'] = (
    df_US['tags']
    .replace('[none]', '')            # replace '[none]' with empty string
    .fillna('')                       # handle missing values
    .apply(lambda x: 0 if x == '' else len(x.split('|')))  # count tags
)

# Quick check
df_US[['tags', 'Num_tags']].head()

,tags,Num_tags
0,SHANtell martin,1
1,"last week tonight trump presidency|""last week ...",4
2,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",23
3,"rhett and link|""gmm""|""good mythical morning""|""...",27
4,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",14


#### 6. Drop comments_disabled, ratings_disabled and video_error_or_removed columns (qianmian)

This checks whether it’s useful (e.g., if almost all are False, maybe drop it).

Since less than 2% of the data for these boolean columns are False, we decided to not use these columns in our anaylsis and drop them.

In [20]:
# Frequency table for comments_disabled, ratings_disabled and video_error_or_removed columns

freq_booleans = pd.DataFrame({
    'comments_disabled': df_US['comments_disabled'].value_counts(),
    'ratings_disabled': df_US['ratings_disabled'].value_counts(),
    'video_error_or_removed': df_US['video_error_or_removed'].value_counts()
}).fillna(0).astype(int)

print("Combined frequency table for columns:")
print(freq_booleans)

freq_percent = df_US[['comments_disabled', 'ratings_disabled', 'video_error_or_removed']] \
    .apply(lambda col: col.value_counts(normalize=True) * 100) \
    .round(2)

print("Percentage of True/False for each column:")
print(freq_percent)

Combined frequency table for columns:
       comments_disabled  ratings_disabled  video_error_or_removed
False              40316             40780                   40926
True                 633               169                      23
Percentage of True/False for each column:
       comments_disabled  ratings_disabled  video_error_or_removed
False              98.45             99.59                   99.94
True                1.55              0.41                    0.06


In [21]:
# Columns are dropped
df_US = df_US.drop(['comments_disabled', 'ratings_disabled', 'video_error_or_removed'], axis=1)

In [22]:
# Prints updated dataframe
print("Updated DataFrame (after dropping boolean columns):")
df_US.info()
print("\nPreview of updated DataFrame:")
print(df_US.head())

Updated DataFrame (after dropping boolean columns):
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40949 entries, 0 to 40948
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   video_id       40949 non-null  object        
 1   trending_date  40949 non-null  datetime64[ns]
 2   title          40949 non-null  object        
 3   channel_title  40949 non-null  object        
 4   category_id    40949 non-null  int64         
 5   publish_time   40949 non-null  datetime64[ns]
 6   tags           40949 non-null  object        
 7   views          40949 non-null  int64         
 8   likes          40949 non-null  int64         
 9   dislikes       40949 non-null  int64         
 10  comment_count  40949 non-null  int64         
 11  category_name  40949 non-null  object        
 12  Num_tags       40949 non-null  int64         
dtypes: datetime64[ns](2), int64(6), object(5)
memory usage: 4.1+ MB

Prev

In [23]:
# Makes new cleaned CSV file
df_US.to_csv("cleaned_USvideos_all.csv", index=False)

#### NOTE

Called df_US.info() to check that all columns' dtypes match what they are supposed to be, and printed the dataset for checking.